<font size="30">Análise Comparativa dos Dados</font>

## 1. Obtenção de Dados

### 1.1 Importação de bibliotecas

In [2]:
import pandas as pd
from IPython.display import display, Markdown, HTML
import joblib
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import classification_report
from sklearn.exceptions import UndefinedMetricWarning

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

### 1.2 Importação dos Dados

Nessa etapa obteremos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pre-processamento, pois serão utilizados além de serem necessários para visualização

In [3]:
caminho = '../data/raw/Orange_Quality_Data.csv'
laranjas = pd.read_csv(caminho) #Obtendo o dataset

laranjas

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,2.0,10,4.0,Orange,Valencia,N,4.0
1,8.2,220,10.5,3.4,3.0,14,4.5,Deep Orange,Navel,N,4.5
2,6.8,150,14.0,3.0,1.0,7,5.0,Light Orange,Cara Cara,N,5.0
3,9.0,250,8.5,3.8,4.0,21,3.5,Orange-Red,Blood Orange,N,3.5
4,8.5,210,11.5,3.3,2.5,12,5.0,Orange,Hamlin,Y (Minor),4.5
...,...,...,...,...,...,...,...,...,...,...,...
236,8.0,194,10.9,3.6,5.0,13,1.0,Orange-Red,Tangerine,Y (Scars),5.0
237,7.4,275,8.5,3.5,5.0,20,5.0,Light Orange,Minneola (Hybrid),N,4.0
238,7.5,196,15.7,3.0,3.0,13,3.0,Deep Orange,Temple,Y (Minor Insect Damage),5.0
239,7.2,251,9.8,4.3,3.0,23,1.0,Light Orange,Moro (Blood),Y (Minor Insect Damage),3.0


In [4]:
dicionario = pd.read_csv("../data/external/dicionario.csv")
dicionario

,variavel,descrição,tipo,subtipo
0,Size (cm),Tamanho da fruta em cm,Quantitativa,Contínua
1,Weight (g),Peso da fruta em g,Quantitativa,Contínua
2,Brix (Sweetness),Nível de doçura,Quantitativa,Contínua
3,pH (Acidity),Nível de acidez em pH,Quantitativa,Contínua
4,Softness (1-5),Maciez de 1-5,Quantitativa,Discreta
5,HarvestTime (days),Dias desde a colheita,Quantitativa,Discreta
6,Ripeness (1-5),Maduração de 1-5,Quantitativa,Discreta
7,Color,Cor da laranja,Qualitativa,Nominal
8,Variety,Variedade da laranja,Qualitativa,Nominal
9,Blemishes (Y/N),Defeito,Qualitativa,Nominal


## 2. Preparação de dados

Este bloco de código separa as colunas qualitativas e quantitativas para uma análise adequada, define as variáveis de entrada (X) e a variável alvo (y) para um problema de regressão. Esta estrutura é essencial para o pré-processamento dos dados e para a construção de modelos de aprendizado de máquina.

In [7]:
# Aqui, a variável target_column é definida como 'Quality (1-5)'. Esta é a variável que queremos prever, o que caracteriza um problema de regressão.
target_column = 'Quality (1-5)'

# Usa a função query para filtrar o dicionário, selecionando apenas as variáveis que são do tipo 'Qualitativa' e que não são a variável alvo ('Quality (1-5)').
# qualitative_columns contém uma lista de todas as colunas qualitativas que não são a variável alvo.
qualitative_columns = (
    dicionario
    .query("tipo == 'Qualitativa' and variavel != @target_column")
    .variavel
    # Seleciona a coluna 'variavel' do resultado do query, que contém os nomes das variáveis qualitativas.
    .to_list()
)

# quantitative_columns contém uma lista de todas as colunas quantitativas que não são a variável alvo.
quantitative_columns = (
    dicionario
    .query("tipo == 'Quantitativa' and variavel != @target_column")
    .variavel
    .to_list()
)



X = laranjas.drop(columns=[target_column], axis=1)
# os dados de entrada X são definidos removendo a coluna de qualidade 'Quality (1-5)' do dataframe laranjas.

y = laranjas[target_column]
# y agora contém os valores da coluna 'Quality (1-5)'

In [21]:
quantitative_columns


['Size (cm)',
 'Weight (g)',
 'Brix (Sweetness)',
 'pH (Acidity)',
 'Softness (1-5)',
 'HarvestTime (days)',
 'Ripeness (1-5)']

---

Este bloco de código configura pipelines para o pré-processamento de variáveis qualitativas e quantitativas em um dataset, lidando com dados faltantes, codificação de variáveis categóricas, e normalização. Esses pipelines são então combinados em um ColumnTransformer que aplica o pré-processamento de forma adequada com base no tipo de cada coluna. O objetivo é preparar os dados para análise ou treinamento de modelos de machine learning de maneira consistente e eficaz.

In [23]:
# tratamento de dados discrepantes
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), 
    # Preenche os valores faltantes com o valor mais frequente da coluna. Isso é comum em variáveis nominais, onde substituímos valores ausentes por uma categoria comum.

    ('encoding', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='infrequent_if_exist')),
    # Codifica variáveis categóricas em uma forma numérica utilizando a técnica de One-Hot Encoding.
    
    ('normalization', StandardScaler())
    # Normaliza as variáveis contínuas para que tenham média zero e variância um, padronizando os dados e facilitando o treinamento de modelos.
])


continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

Combina os pipelines de pré-processamento para aplicar transformações diferentes a colunas diferentes no mesmo dataset.
preprocessor = ColumnTransformer([
    ('qualitative', nominal_preprocessor, qualitative_columns),
    ('quantitative', continuous_preprocessor, quantitative_columns)
])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('qualitative', OneHotEncoder(handle_unknown='ignore'), categorical_features),
#         ('quantitative', StandardScaler(), numeric_features)
#     ]
# )

## 3. Seleção de modelos

Neste experimento, iremos analisar quatro modelos de regressão para prever a variável alvo, utilizando um método de validação cruzada e otimização de hiperparâmetros. Os modelos selecionados para a análise são:

- **Regressão Linear (Linear Regression)**
- **Árvore de Decisão (Decision Tree Regressor)**
- **Floresta Aleatória (Random Forest Regressor)**
- **K-Vizinhos Mais Próximos (K-Neighbors Regressor)**

Cada modelo será avaliado com diferentes hiperparâmetros para identificar a melhor configuração e o melhor desempenho possível para o problema em questão. O objetivo é encontrar o modelo que melhor se ajusta aos dados de treinamento e tem o menor erro de previsão.


 **Configuração do Experimento**

- **Validação Cruzada**: Usaremos validação cruzada com divisão dos dados em três partes para a análise comparativa e duas partes para a busca de hiperparâmetros.

- **Divisão dos Dados**: 30% dos dados serão reservados para teste e 70% para treinamento.

- **Estado Aleatório**: Para garantir a reprodutibilidade dos resultados, usaremos um estado aleatório fixo (42).

- **Métrica Principal**: Usaremos o Erro Médio Absoluto Negativo (neg_mean_absolute_error) como a principal métrica para avaliação dos modelos.



**Métricas de Avaliação**

Serão utilizadas as seguintes métricas para avaliar o desempenho dos modelos:

- **Erro Médio Absoluto Negativo (neg_mean_absolute_error)**: Mede a diferença média absoluta entre os valores previstos e os valores observados, onde menores valores indicam melhor desempenho.
- **R² (R-squared)**: Avalia a proporção da variabilidade da variável dependente que é explicada pelo modelo. Valores mais próximos de 1 indicam melhor ajuste.
- **Erro Máximo (max_error)**: Mede o maior erro absoluto entre as previsões e os valores observados. Menores valores indicam melhores previsões.
- **Variância Explicada (explained_variance)**: Mede a proporção de variação explicada pelo modelo. Valores próximos de 1 indicam uma melhor explicação da variabilidade dos dados.

In [29]:
# Configurações do experimento
n_splits_comparative_analysis = 3
# Define o número de divisões para a análise comparativa usando validação cruzada. Nesse caso, os dados serão divididos em 3 subconjuntos para realizar testes de validação cruzada.
n_folds_grid_search = 2
# Define o número de partes em que os dados serão divididos para a busca em grade (Grid Search). Aqui, os dados serão divididos em 2 partes para testar diferentes combinações de hiperparâmetros.
test_size = 0.3
# Proporção do conjunto de dados que será reservado para o teste. Neste caso, 30% dos dados serão usados para teste e 70% para treinamento.
random_state = 42
# Define um valor fixo para o estado aleatório para garantir a reprodutibilidade dos resultados. Usar o mesmo random_state permite que os mesmos dados sejam usados em cada execução.
scoring = 'neg_mean_absolute_error'
# Especifica a métrica principal para avaliação do modelo durante a Grid Search. O neg_mean_absolute_error é a versão negativa do erro médio absoluto, 
# usado porque métricas de erro negativo são preferíveis em certos contextos de otimização.



# Lista de métricas que serão usadas para avaliar o desempenho dos modelos
metrics = ['neg_mean_absolute_error', 'r2', 'max_error', 'explained_variance']

# Lista de modelos de regressão a serem testados. Cada entrada consiste em:
# O nome do modelo.
# O objeto do modelo (e.g., LinearRegression(), DecisionTreeRegressor()).
# Dicionário de hiperparâmetros para a busca em grade. Exemplo: criterion, max_depth para árvores de decisão e floresta aleatória.
models = [
    ('Linear Regression', LinearRegression(), {}),
    ('Decision Tree', DecisionTreeRegressor(random_state=random_state), {'criterion': ['squared_error', 'absolute_error'], 'max_depth': [3, 6, 8]}),
    ('Random Forest', RandomForestRegressor(random_state=random_state), {'criterion': ['squared_error', 'absolute_error'], 'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
    ('K Neighbors', KNeighborsRegressor(), {'n_neighbors': [1, 3, 5, 10]}),
]



results = pd.DataFrame({})
# Cria um DataFrame vazio para armazenar os resultados das validações cruzadas.
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
# Configura a validação cruzada para a Grid Search com 2 divisões dos dados.
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
# Configura a validação cruzada para a análise comparativa, que mistura os dados aleatoriamente e divide em 3 conjuntos, reservando 30% para teste.


for model_name, model_object, model_parameters in models:
# Itera sobre cada modelo na lista de modelos definidos.
    print(f"Running {model_name}...")
    
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=2,
        cv=cross_validate_grid_search
    )
    # Configura a busca em grade para o modelo atual. 
    # Testa diferentes combinações de hiperparâmetros usando a métrica de avaliação scoring. 
    # Utiliza 2 núcleos de CPU (n_jobs=2) e a configuração de validação cruzada especificada.
    
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    # Cria um pipeline para o modelo atual, incluindo o pré-processador de dados (preprocessor) e o modelo com a busca em grade (model_grid_search).
    
    scores = cross_validate(
            estimator=approach,
            X=X,
            y=y,
            cv=cross_validate_comparative_analysis,
            n_jobs=2,
            scoring=metrics
        )
    # Realiza a validação cruzada do pipeline completo com os dados de entrada X e a variável alvo y. 
    #Utiliza a configuração de validação cruzada para análise comparativa e avalia o modelo com as métricas especificadas.

    scores['model_name'] = [model_name] * n_splits_comparative_analysis
    # Adiciona uma coluna ao DataFrame de resultados para indicar o nome do modelo testado.
    df_scores = pd.DataFrame(scores)
    # Converte os resultados da validação cruzada em um DataFrame.
    df_scores = df_scores.drop(columns=['model_name'])
    # Remove a coluna model_name para calcular a média (mean) e o desvio padrão (std) das métricas de desempenho para o modelo atual.
    df_scores = df_scores.agg(['mean', 'std'])
    # Exibe os resultados de desempenho (média e desvio padrão das métricas) para o modelo atual.
    
    
    print(f"Results for {model_name}:")
    display(df_scores)
    #Exibe os resultados de desempenho (média e desvio padrão das métricas) para o modelo atual.
    
    results = pd.concat([results, pd.DataFrame(scores)], ignore_index=True)
    # Concatena os resultados atuais ao DataFrame geral de resultados.

# Mostrar resultados finais
print("Final results:")
display(results)

Running Linear Regression...
Results for Linear Regression:


/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_neg_mean_absolute_error,test_r2,test_max_error,test_explained_variance
mean,0.054607,0.015326,-0.622551,0.291379,-2.065229,0.308029
std,0.006347,0.002269,0.052361,0.081479,0.421525,0.090229


Running Decision Tree...
Results for Decision Tree:


/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


...

Running K Neighbors...
Results for K Neighbors:


/mnt/c/Users/bruno/AvantiBootcamp/equipe7-2024.2/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_neg_mean_absolute_error,test_r2,test_max_error,test_explained_variance
mean,0.076132,0.015090,-0.551446,0.357886,-2.716667,0.406301
std,0.013504,0.001131,0.075134,0.126591,0.246644,0.137908


Final results:


,fit_time,score_time,test_neg_mean_absolute_error,test_r2,test_max_error,test_explained_variance,model_name
0,0.048153,0.017530,-0.562700,0.384896,-1.633459,0.410536,Linear Regression
1,0.060841,0.015452,-0.659893,0.253553,-2.475703,0.272916,Linear Regression
2,0.054828,0.012997,-0.645061,0.235687,-2.086525,0.240636,Linear Regression
3,0.106801,0.012230,-0.828767,-0.731875,-4.000000,-0.440891,Decision Tree
4,0.102231,0.008916,-0.630137,0.284882,-3.500000,0.299179,Decision Tree
5,0.076233,0.013409,-0.561644,0.377519,-2.500000,0.387016,Decision Tree
6,1.272409,0.014747,-0.478955,0.506328,-1.865411,0.518627,Random Forest
7,1.283031,0.013696,-0.542576,0.442594,-3.052330,0.472407,Random Forest
8,1.002519,0.011070,-0.475690,0.599842,-2.025084,0.600091,Random Forest
9,0.085963,0.016377,-0.475342,0.455408,-2.550000,0.474528,K Neighbors


In [19]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("### 3.3 Resultados gerais e discussão"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 3.3 Resultados gerais e discussão

Para determinar qual classificador obteve o melhor resultado no seu trabalho de ciência de dados, vamos analisar os seguintes indicadores na tabela:

- test_neg_mean_absolute_error (Erro Absoluto Médio Negativo) - Quanto menor, melhor.
- test_r2 (Coeficiente de Determinação R²) - Quanto mais próximo de 1, melhor.
- test_max_error (Erro Máximo) - Quanto menor, melhor.
- test_explained_variance (Variância Explicada) - Quanto mais próximo de 1, melhor.

---

O classificador Random Forest obteve o melhor desempenho geral. Os principais motivos incluem:

- Melhor desempenho em test_neg_mean_absolute_error, indicando que ele teve o menor erro absoluto médio.
- Maior valor de test_r2, indicando que explicou melhor a variabilidade dos dados.
- Melhor test_explained_variance, mostrando que capturou mais da variância nos dados em comparação com os outros modelos.

Esses resultados sugerem que o Random Forest tem uma melhor capacidade preditiva e generalização para este conjunto de dados em particular, comparado aos outros modelos avaliados.